[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ryansteed/ieat/blob/master/notebooks/experiments.ipynb)

# Image GPT Bias
**Image Embedding Association Test**

Ryan Steed

This script adapted from https://colab.research.google.com/github/apeguero1/image-gpt/blob/master/Transformers_Image_GPT.ipynb.


In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

%cd /Users/steed/caliskan/git/image-gpt-bias

/Users/steed/caliskan/git/image-gpt-bias


## Download Model

In [ ]:
!nvidia-smi #OpenAI says you need 16GB GPU for the large model, but it may work if you lower n_sub_batch on the others.

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import urllib

# don't share these to Github or elsewhere!
u = "your_username"
p = urllib.parse.quote("your_password")
!git clone --recurse-submodules -j8 https://$u:$p@github.com/$u/image-gpt-bias.git
%cd image-gpt-bias

Cloning into 'image-gpt-bias'...
remote: Invalid username or password.
fatal: Authentication failed for 'https://github.com/your_username/image-gpt-bias.git/'
[Errno 2] No such file or directory: 'image-gpt-bias'
/Users/steed/caliskan/git/image-gpt-bias


In [ ]:
# note - you may need to restart the kernel for these installations to take effect
!pip install -e .
!pip install -e weat

Obtaining file:///content/image-gpt-bias
     |████████████████████████████████| 1.0MB 8.1MB/s 
     |████████████████████████████████| 1.1MB 31.8MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 3.0MB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=39f329199919716d85b4b368f2f367d7302eac9cc8fbdabedb3fbb772ed215de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Running setup.py develop for ieat
Obtaining file:///content/image-gpt-bias/weat
  Running setup.py develop for weat


In [ ]:
model_sizes = ["s", "m", "l"] #small medium large, xl not available
model_size = "l"
models_dir = "models"
color_clusters_dir = "clusters"
n_px = 32
depth = 152
width = 3
sk = 1

In [ ]:
# download the model - skip if already downloaded
!python image-gpt/download.py --model {model_size} --ckpt 1000000 --clusters --download_dir {models_dir}/{model_size}
!python image-gpt/download.py --clusters --download_dir {color_clusters_dir}

## IATs

### Quick Summary

In [ ]:
from ieat.api import test_all

import pandas as pd
import numpy as np

results = test_all(
    {
        'igpt-logit': (model_size,models_dir,color_clusters_dir,n_px),
        'igpt': (model_size,models_dir,color_clusters_dir,n_px),
        'simclr': (depth, width, sk)
    }
)

# igpt-logit #
## Insect-Flower ##


2020-10-01 10:37:37,457 [82043] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


## Weapon ##
## Weapon (Modern) ##


2020-10-01 10:37:37,563 [82043] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value
2020-10-01 10:37:37,730 [82043] WARNING  root:202: [JupyterRequire] Equalities contributed 1/10000 to p-value


## Native ##
## Asian ##


...

2020-10-01 10:37:39,250 [82043] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


## Race ##
## Arab-Muslim ##


2020-10-01 10:37:39,560 [82043] WARNING  root:202: [JupyterRequire] Equalities contributed 1/924 to p-value


## Age ##
## Gender-Science ##
## Gender-Career ##
## Race-WMBM ##


AssertionError: There is no file at models/l/model.ckpt-1000000.index

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results).transpose()
results_df.columns = ["X", "Y", "A", "B", "d", "p", "n_t", "n_a"]
for c in results_df.columns[:4]:
    results_df[c] = results_df[c].str.split("/").str[-1]
results_df["sig"] = ""
for l in [0.10, 0.05, 0.01]:
    results_df.sig[results_df.p < l] += "*"
# results_df[results_df.index.get_level_values(1) == "simclr"]
results_df

In [ ]:
# Generate latex table output
experiments = (
    "Insect-Flower",
    "Gender-Science",
    "Gender_Career",
    "Skin-Tone",
    "Race",
    "Weapon",
    "Weapon (Modern)",
    "Native",
    "Asian",
    "Weight",
    "Religion",
    "Sexuality",
    "Disability",
    "Arab-Muslim",
    "Age"
)
latex_df = results_df
latex_df.index = latex_df.index.set_names(["Test", "Model"])
latex_df = latex_df.sort_index(level=["Test", "Model"])
latex_df["d"] = latex_df.d.apply(lambda x: '{0:.2f}'.format(x)).astype(str) + latex_df.sig.astype(str)
latex_df = latex_df.drop(columns=["sig"])
latex_df.columns = [f"${col}$" for col in latex_df.columns]
latex_df[latex_df.index.get_level_values("Test").isin(experiments)]\
    .drop(columns=["$X$", "$Y$", "$A$", "$B$"]).to_latex('output/results.tex')
original_iats = {
    # IAT, d, r
    ["Age", 
}
latex_df.droplevel("Model").groupby("Test").head(1)\
    .drop(columns=["$d$", "$p$", "$n_t$", "$n_a$"]).to_latex('output/iats.tex')
stimuli = pd.read_csv("data/stimuli.csv")
stimuli.dropna(axis=0, subset=["Word", "Source"])\
    .drop(columns="Collected (N=n)").set_index(["IAT", "Category"]).to_latex("output/stimuli.tex")

### Detail View

#### Weapons

In [ ]:
from ieat.api import test

test(
    "data/experiments/weapon/black",
    "data/experiments/weapon/white",
    "data/experiments/weapon/tool",
    "data/experiments/weapon/weapon",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Native

In [ ]:
test(
    "data/experiments/native/euro",
    "data/experiments/native/native",
    "data/experiments/native/us",
    "data/experiments/native/world",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Asian

In [ ]:
test(
    "data/experiments/asian/european-american",
    "data/experiments/asian/asian-american",
    "data/experiments/asian/american",
    "data/experiments/asian/foreign",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Insect-Flower

In [ ]:
from ieat.api import test

test(
    "data/experiments/insect-flower/flower",
    "data/experiments/insect-flower/insect",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Weight

In [ ]:
from ieat.api import test

test(
    "data/experiments/weight/thin",
    "data/experiments/weight/fat",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Skin-Tone

In [ ]:
from ieat.api import test

test(
    "data/experiments/skin-tone/light",
    "data/experiments/skin-tone/dark",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Disability

In [ ]:
from ieat.api import test

test(
    "data/experiments/disabled/disabled",
    "data/experiments/disabled/abled",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Presidents

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/kennedy",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/clinton",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/bush",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/lincoln",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Religion

In [ ]:
from ieat.api import test

test(
    "data/experiments/religion/christianity",
    "data/experiments/religion/judaism",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Science

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/science",
    "data/experiments/gender/liberal-arts",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Career

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/career",
    "data/experiments/gender/family",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Sexuality

In [ ]:
from ieat.api import test

test(
    "data/experiments/sexuality/gay",
    "data/experiments/sexuality/straight",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Race

In [ ]:
from ieat.api import test

test(
    "data/experiments/race/african-american",
    "data/experiments/race/european-american",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Arab-Muslim

In [ ]:
from ieat.api import test

test(
    "data/experiments/arab-muslim/other-people",
    "data/experiments/arab-muslim/arab-muslim",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Age

In [ ]:
from ieat.api import test

test(
    "data/experiments/age/young",
    "data/experiments/age/old",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

### Download Cached Embeddings

In [ ]:
# to download from colab
from google.colab import files

!zip -r embeddings_colab.zip embeddings
files.download("embeddings_colab.zip")